In [11]:
import pandas as pd
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, HoverTool, NumeralTickFormatter

############################################
def getMA(series, maperiod ,Case):
    
    
    if Case==1:
        # Use exponential moving average
        alpha=1/(maperiod) ###RMA
        ma = series.ewm(alpha=alpha,adjust=True, min_periods = maperiod).mean()
        ###Otherway
        #ma = series.ewm(com = maperiod - 1, adjust=True, min_periods = maperiod).mean()
     
    elif Case==2:
        alpha=2/(maperiod+1) ###EMA
        ma = series.ewm(alpha=alpha,adjust=True, min_periods = maperiod).mean()
        
    elif Case==3:
        # Use simple moving average
        ma = series.rolling(window = maperiod).mean()
        
    return ma
# #########################################
def ATR(maperiod):
    global df
    for i in range(1,len(df)):
        df.loc[i,'high_low']=df.loc[i,'High']-df.loc[i,'Low']
        df.loc[i,'high_preClose']=abs(df.loc[i,'High']-df.loc[i-1,'Close'])
        df.loc[i,'low_preClose']=abs(df.loc[i,'Low']-df.loc[i-1,'Close'])
        df.loc[i,'trueRange']=max(df.loc[i,'high_low'],df.loc[i,'high_preClose'],df.loc[i,'low_preClose'])
        df['ATR']=getMA(df['trueRange'],maperiod,1)

############################################
def candlestick_plot(df, name):
   
    # Select the datetime format for the x axis depending on the timeframe
    xaxis_dt_format = '%d %b %Y'
#     if df['Date'][0].hour > 0:
#         xaxis_dt_format = '%d %b %Y, %H:%M:%S'
#     print(df)
    fig = figure(sizing_mode='stretch_both',
                 tools="xpan,xwheel_zoom,reset,save",
                 active_drag='xpan',
                 active_scroll='xwheel_zoom',
                 x_axis_type='linear',
                 # x_range=Range1d(df.index[0], df.index[-1], bounds="auto"),
                 title=name
                 )
    fig.yaxis[0].formatter = NumeralTickFormatter(format="$5.3f")
    inc = df.Close > df.Open
    dec = ~inc

    # Colour scheme for increasing and descending candles
    inc_color = '#096916'
    dec_color = '#cc0606'

    w = 0.5   #width
    inc_source = ColumnDataSource(data=dict(
        x1=df.index[inc],
        top1=df.Open[inc],
        bottom1=df.Close[inc],
        high1=df.High[inc],
        low1=df.Low[inc],
        Date1=df.Date[inc]
    ))

    dec_source = ColumnDataSource(data=dict(
        x2=df.index[dec],
        top2=df.Open[dec],
        bottom2=df.Close[dec],
        high2=df.High[dec],
        low2=df.Low[dec],
        Date2=df.Date[dec]
    ))
    print(inc_source.data)
    print(dec_source.data)


    # Plot candles
    # High and low
    fig.segment(
                        x0='x1',
                        y0='high1',
                        x1='x1', 
                        y1='low1', 
                        source=inc_source, 
                        color=inc_color,
                        line_width=2,
                    )
    fig.segment(
                        x0='x2',
                        y0='high2', 
                        x1='x2', y1='low2', 
                        source=dec_source, 
                        color=dec_color,
                        line_width=2,
                    )

    # Open and close
    r1 = fig.vbar(
                        x='x1',
                        width=w, 
                        top='top1', 
                        bottom='bottom1', 
                        source=inc_source,
                        fill_color=inc_color, 
                        line_color=inc_color
                    )
    r2 = fig.vbar(
                        x='x2', 
                        width=w, 
                        top='top2', 
                        bottom='bottom2',
                        source=dec_source,
                        fill_color=dec_color, 
                        line_color=dec_color
                    )

    # Add on extra lines (e.g. moving averages) here
    # fig.line(df.index, <your data>)
    p1 = figure(
    x_range=fig.x_range,
    plot_height=120,
    plot_width=1500, 
    title="ATR",
    tools="",
    x_axis_location=None,
    
    )
    p1.xgrid.grid_line_color=None

    p1.line(
        df.index,
        df['ATR'],
        line_width=2,
        color="orange"
    )
    p1.line(
        df.index,
        df['ATR_MSD'],
        color="blue"
    )
    p1.line(
        df.index,
        df['Close_MSD'],
        color="green"

    )

    
    
    
    # Add on a vertical line to indicate a trading signal here
    # vline = Span(location=df.index[-<your index>, dimension='height',
    #              line_color="green", line_width=2)
    # fig.renderers.extend([vline])

    # Add date labels to x axis
    fig.xaxis.major_label_overrides = {
        i: date.strftime(xaxis_dt_format) 
        for i, date in enumerate(pd.to_datetime(df["Date"]))
    }

    
    # Set up the hover tooltip to display some useful data
    fig.add_tools(HoverTool(
        renderers=[r1],
        tooltips=[
            ("Open", "$@top1"),
            ("High", "$@high1"),
            ("Low", "$@low1"),
            ("Close", "$@bottom1"),
            ("Date", "@Date1"),
        ],
        formatters={
            'Date1': 'datetime',
         }
    ))

    fig.add_tools(HoverTool(
        renderers=[r2],
        tooltips=[
            ("Open", "$@top2"),
            ("High", "$@high2"),
            ("Low", "$@low2"),
            ("Close", "$@bottom2"),
            ("Date", "@Date2")
        ],
        formatters={
            'Date2': 'datetime'
        }
    ))
   
    # JavaScript callback function to automatically zoom the Y axis to
    # view the data properly
    source = ColumnDataSource({'Index': df.index, 'High': df.High, 'Low': df.Low})
    callback = CustomJS(args={'y_range': fig.y_range, 'source': source}, code='''
    
            clearTimeout(window._autoscale_timeout);
            
            var Index = source.data.Index,
            Low = source.data.Low,
            High = source.data.High,
            start = cb_obj.start,
            end = cb_obj.end,
            min = Infinity,
            max = -Infinity;

        for (var i=0; i < Index.length; ++i) {
            if (start <= Index[i] && Index[i] <= end) {
                max = Math.max(High[i], max);
                min = Math.min(Low[i], min);
            }
        }
        var pad = (max - min) * .05;

        window._autoscale_timeout = setTimeout(function() {
            y_range.start = min - pad;
            y_range.end = max + pad;
        });
        
    ''')

    # Finalise the figure
    fig.x_range.js_on_change('start', callback)
    show(column(fig, p1))
    from IPython.display import HTML
    HTML("./German.html")

# Main function
if __name__ == '__main__':
    # Read CSV
    df = pd.read_csv("German.csv")



    # Reverse the order of the dataframe - comment this out if it flips your chart
    df = df[::-1]
    df.index = df.index[::-1]
    #print(df)
    
    df=df[['Date','Open','High','Low','Price','Vol.','Change %']]
    
    df.rename(columns = {'Price':'Close'}, inplace = True)
    print(df)

    N=14 ### its user defined 
    ATR(N)
    df=df[['Date','Open','High','Low','Close','Vol.','Change %','ATR']]
    df['Close_MSD']=df['Close'].rolling(N).std()
    df['ATR_MSD']=df['ATR'].rolling(N).std()
    print(df.head(40))    
    # Trim off the unnecessary bit of the minute timeframe data - can be unnecessary
    # depending on where you source your data
#     if '-04:00' in df['Date'][0]:
#         df['Date'] = df['Date'].str.slice(0, -6)

    # Convert the dates column to datetime objects
    #df["Date"] = pd.to_datetime(df["Date"], format='%Y-%m-%d %H:%M:%S')  # Adjust this
    df["Date"] = pd.to_datetime(df["Date"], format='%b %d, %Y')
    output_file("German_Plot.html")
    candlestick_plot(df, "German_Plot")


              Date    Open    High     Low   Close     Vol. Change %
0     Jan 02, 2015  155.84  156.57  155.74  156.48  337.40K    0.45%
1     Jan 05, 2015  156.51  156.58  156.20  156.22  593.49K   -0.17%
2     Jan 06, 2015  156.40  157.20  156.29  156.92  735.02K    0.45%
3     Jan 07, 2015  156.85  157.26  156.51  156.73  759.12K   -0.12%
4     Jan 08, 2015  156.61  156.75  156.27  156.33  676.11K   -0.26%
...            ...     ...     ...     ...     ...      ...      ...
1900  Jun 09, 2022  148.69  149.43  147.32  147.80    1.04M   -0.77%
1901  Jun 10, 2022  147.74  148.51  146.61  146.98  956.29K   -0.55%
1902  Jun 13, 2022  146.71  147.19  144.19  145.18    1.10M   -1.22%
1903  Jun 14, 2022  144.65  145.58  142.29  143.59    1.26M   -1.10%
1904  Jun 15, 2022  142.73  145.51  142.25  144.61    1.29M    0.71%

[1905 rows x 7 columns]
            Date    Open    High     Low   Close     Vol. Change %       ATR  \
0   Jan 02, 2015  155.84  156.57  155.74  156.48  337.40K    0.45% 